# Sentence categorisation using TF-IDF with k-means

The dataset was found here: https://wortschatz.uni-leipzig.de/en/download/French
It contains a mix of one million french sentences from various sources.


The app will have many sentences in the target language that the user will see one after the other in increasing difficulty order. However, it would be nice if the user had the option to filter down the sentences they see to certain categories. For example, if they would like to practice vocabulary specific to the home, or sports. Instead of manually categorising what will eventually be thousands of sentences, I need to create a ml model to automatically place new sentences into a category as accurately as possible.

There are a few supervised methods I could use such as naive bayes, support vector machines as well as deep learning models. However, I'll probably get the best results by using google's pre-trained BERT model as a base to encode the sentences and training a few fully-connected layers at the end for my custom classifyer. Computational cost is not so much of a concern as I'll only be running the classifier once on the dataset to generate a category column for the database. Although BERT was originally trained on an english corpus, there's another model called Multilingual BERT that has been trained on Wikipedia in 104 different languages, which would be well-suited to this application.

However, I found it pretty hard to find a labeled ground-truth dataset for sentence-level subject classification, especially for foreign languages. I could create one myself but making an unbiased labelled dataset with enough samples in each category would be pretty difficult and probably not worth the time investment in this case, so in the end I've decided to use an unsupervised approach and infer categories. I need to pick between clustering and topic modelling.

For this application, I want the sentences to be clustered by topic and not something else like sentiment. Representing the sentences using a bag-of-words model would group sentences together based on the words they contain, which would be more likely to result in topic-based clusters. Better yet, I could use TF-IDF, which, like bag-of-words, assigns a score based on the number of times a word appears in the document but is counterbalanced by the number of sentences in which it is present. Words like ‘this’, ’are’, that are commonly present in all the sentences are not given a very high rank. However, a word that is present too many times in a few of the sentences will be given a higher rank as it might be indicative of its context.

Once I have fit the k-means model I can use it to predict which category new sentences belong to.

In [2]:
import os
import re
import math
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
#from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
#filepath = os.path.join("./fra_mixed_2009_1M", "fra_mixed_2009_1M-sentences.txt")

# Train on cleaned dataset of around 850,000 french sentences, capped to 200 characters max
filepath = os.path.join("..", "french_sentences.csv")

df = pd.read_csv(filepath, delimiter='\t', header=None)
df.columns = ["id", "sentence"]

In [ ]:
df.info()

In [ ]:
df.head()

# Cleaning the data

Delete any potential duplicates

In [ ]:
# Let's see if there are any duplicates in the dataset
df[df["sentence"].duplicated(keep=False)].sort_values("sentence").head(8)

In [ ]:
# Remove all duplicates from the dataframe
df = df.drop_duplicates("sentence")

# NLP 

In [ ]:
# First we need to get a list of stop words in french. These are common filler words that provide almost no useful information
# and occur frequently in most documents. Stop words can have a disproportionate influence on the overall representation of
# the document, which can be detrimental to the performance of TF-IDF. To mitigate this we need to remove stop words before
# calculating TF-IDF vectors

punc = ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}',"%"]

# Get French stop words
french_stop_words = stopwords.words('french')

# Combine with standard punctuation to get the comprehensive list of stop words. Convert stop words to a set
# to ensure no duplicates
stop_words = set(french_stop_words).union(punc)

# convert back to list
stop_words = list(stop_words)

print(stop_words)

There are lots of possible numerical values that can occur within a text, which means that a large fraction of the unique values could be numerical or at least contain strings of numbers. This can add noise to the data and lead to a sparse dataset and affect model performance, so I use a custom token pattern that ignores all words containing any numerical value. I might modify this in the future as it excludes important information such as dates, but as an initial proof-of-concept it's probably not too much of an issue.

In [ ]:
# Get the raw sentences from the dataframe
sentences = df['sentence'].values

# Create a custom token pattern that matches words containing only letters
# The pattern \b\w+\b matches whole words, and [^\W\d_] matches any character that is not a non-word character, digit, or underscore
token_pattern = r'\b[^\W\d_]+\b'

# Create the TF-IDF vectorizer object and fit to our dataset to generate vector encodings for each sentence
vectorizer = TfidfVectorizer(stop_words=stop_words, token_pattern=token_pattern)
X = vectorizer.fit_transform(sentences)

In [ ]:
punc = ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}',"%"]
#stop_words = text.ENGLISH_STOP_WORDS.union(punc)
desc = data['headline_text'].values
vectorizer = TfidfVectorizer(stop_words = stop_words)
X = vectorizer.fit_transform(desc)

In [ ]:
word_features = vectorizer.get_feature_names_out()
# Show the unique words that occurred in the training data
print(len(word_features))
print(word_features[:100])

It looks like the data could be further cleaned. For example, I could remove words with three or more of the same letter in a row. For now I'll leave it but I'll probably come back and properly clean the data once I have a preliminary output.

# Stemming and Tokenizing
Stemming is the process of reducing a word into its stem, i.e. its root form. The root form is not necessarily a word by itself, but it can be used to generate words by concatenating the right suffix. For example, the words fish, fishes and fishing all stem into fish, which is a correct word. On the other side, the words study, studies and studying stems into studi, which is not an English word.

Tokenization is breaking the sentence into words and punctuation,

In [ ]:
stemmer = SnowballStemmer('english')
tokenizer = RegexpTokenizer(r'[a-zA-Z\']+')

def tokenize(text):
    return [stemmer.stem(word) for word in tokenizer.tokenize(text.lower())]

In [ ]:
stemmer = SnowballStemmer('french')
tokenizer = RegexpTokenizer(r'[a-zA-Z\']+')

def _tokenize(text: str) -> list[str]:
    """Tokenizes a document into its individual words and punctuation and returns a list of each token's stem.
    """

    return [stemmer.stem(word) for word in tokenizer.tokenize(text.lower())]

# Vectorization with stop words(words irrelevant to the model), stemming and tokenizing

In [ ]:
vectorizer2 = TfidfVectorizer(stop_words = stop_words, tokenizer = _tokenize)
X2 = vectorizer2.fit_transform(sentences)

word_features2 = vectorizer2.get_feature_names_out()
print(len(word_features2))
print(word_features2[:100]) 

In [ ]:
vectorizer3 = TfidfVectorizer(stop_words = stop_words, tokenizer = _tokenize, max_features = 1000) # limit to 1000 most frequent terms in the corpus
X3 = vectorizer3.fit_transform(sentences)
words = vectorizer3.get_feature_names_out()

# K-means clustering

We can choose the appropriate number of clusters with the elbow method, which just plots the percentage of variance explained (ratio of the between-group variance to the total variance) as a function of the number of clusters. The appropriate cluster number is one where adding another cluster doesn't give much better modelling of the data i.e. the marginal gains of adding more clusters begins to drop off and doesn't provide a significantly better classification of the data. This point can be identified by an angle (elbow) in the graph, although this point isn't always easy to identify.

A rule of thumb to estimate the value of k to choose is 𝑘∼SQRT(n/2) where n is the number of points to be clustered.

In [ ]:
# Get the number of data points to be clustered
num_data_points = X3.shape[0]
recommended_k = math.ceil(math.sqrt(num_data_points/2))

print(f"The recommended value of k when clustering {num_data_points} data points is {recommended_k}")

This is obviously far greater than the 10-15 sentence topic categories I would like to use in my app, so the elbow method will be ineffective as choosing values of k between say five and 20 should result in a relatively inverse linear relationship between k and wcss (within-cluster sum of squares).

The purpose of the topic filter is to choose sentences that strongly match a specific topic, and choosing low k values will lead to extremely general categories, which is obviously not what I want.

One solution is to choose a high value for k, say several hundred, and find clusters that are strongly correlated with a particular topic that I want to include as a filter. There will be no clusters that overlap perfectly with any topic, but I can choose a few that correspond well. This is not a perfect method and there will be blind spots in coverage of each topic, but this is a low-stakes application so that's not too much of an issue. This will lead to most of the sentences being not being used in any topic filter whatsoever, but this isn't really a problem since the dataset is so large that each topic may still contain thousands of sentences for the user to learn from. When no filter is applied, I can show the user sentences chosen from the entire dataset.

I'll plot the graph for 100 to 1000 clusters in increments of 100

In [ ]:
from sklearn.cluster import KMeans

wcss = []

for i in range(100,1001, 100):

    # k-means++: initialization method selects initial cluster centroids that increase convergence speed
    # max_iter: max number of iterations for single run of k-means
    # n_init: The number of times k-means will be run with different centroid seeds. Final result taken
    # as the best output in terms of inertia. Since we're running k means lots of times and only plotting
    # a rough graph, n_init can be lower here to save time
    # random_state: Controls random number generator used for centroid initialization.
    # Setting to 0 seeds the random number generator with a fixed value, ensuring reproducibility.
    kmeans = KMeans(n_clusters=i,init='k-means++',max_iter=300,n_init=10,random_state=0)
    kmeans.fit(X3)
    # Append the inertia for this cluster number (inertia is just the within-cluster sum of squares)
    # to list of wcss for all cluster numbers
    wcss.append(kmeans.inertia_)

# Plot wcss against num clusters
plt.plot(range(1,1001),wcss)
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

As is to be expected, for such a large number of datapoints and clusters, the decrease in wcss with new clusters begins to resemble a smooth, exponentially decreasing curve.

Looking at the plot in wcss_clusters.csv, I'll assume the y intercept y0 is around 900,000. Since wcss=665,000 when num clusters c = 1000, and exponential decay is modelled as f(c) = y0*e^(-c/τ), the curve can be modelled by calculating tau.

If you rearrange the above equation you get τ = c/ln(y0/y) so τ=(1000/ln(900,000/665,000)) which roughly gives τ=3304.

But looking at other data points, f(c) doesn't seem to behave like a standard exponentially decaying function, since τ doesn't stay constant as c increases. In fact, every time c increases by 200, τ seems to increase by a factor of around 1.2. Assuming this were a general rule, you could then say that τ and c are related by the following equation: τ = k * 1.2^(c/200).

This means that the actual equation could be f(c) = y0^e-(c/(k * 1.2^(c/200)))

And we can find k by substituting in wcss = 665,000, y0 = 900,000 and c = 1000 to the above equation gives k = 551.

This is obviously based on a very small number of observations but it should give a reasonable estimate for the function linking wcss and number of clusters without having to actually run more simulations, since it already took 2 days to generate that chart.

I'd say a good number of clusters to choose might be the point when wcss decreases by less than 5% by adding another 100 clusters:

(f(c+100)-f(c))/f(c) <= 0.05

According to this arbitrary cut-off point, and assuming my equation is a reasonable model for the relationship between wcss and c, this gives an optimal cluster number of 1386, which I'll round to 1400.





In [ ]:
print(words[300:500])

# 700 Clusters

Now let's fit a k means with our optimal number of clusters

In [ ]:
# Run with more initial seeds 20 to increase the odds of finding good centroids
# Finally, we look at 8 the clusters generated by k-means.
kmeans = KMeans(n_clusters = 8, n_init = 20)
kmeans.fit(X3)

# Display 100 most common words in each cluster
# sort the feature values of all cluster centers in ascending order using the argsort method, and then take the last 100 indices
"""common_words = kmeans.cluster_centers_.argsort()[:,-1:-101:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))
"""

And print some sentences in each cluster

In [ ]:
"""
# Assign each sentence to a cluster
labels = kmeans.predict(X3)

# Group sentences by cluster
clusters = {}
for i, label in enumerate(labels):
    if label not in clusters:
        clusters[label] = []
    clusters[label].append(sentences[i])

# Print sentences in each cluster
for label, cluster_sentences in clusters.items():
    print(f'Cluster {label}:')
    for sentence in cluster_sentences[:10]:
        print(f'  - {sentence}')
"""

# Save model and print out most common words for each cluster to file

For each sentence, I'll add a new column in the database to indicate which cluster it belongs to. Then, once I have identified which clusters correcspond to which topics, I can simply filter the sentences accordingly when the user selects that topic.

Saving the model means I can reuse it later to predict which cluster new sentences will be in without having to re-run the entire model, which would lead to different clusters and require me to redefine which clusters correspond to which topic, and then update the database, which would be tedious.

In [ ]:
import pickle

# Fitted scikit-learn models can be saved using pickle
with open("kmeans.pkl", "wb") as f:
    pickle.dump(kmeans, f)

# Save most common words for each cluster in a txt a file
common_words = kmeans.cluster_centers_.argsort()[:,-1:-101:-1]

with open("cluster_words.txt", "w") as f:

    for num, centroid in enumerate(common_words):
        f.write("Cluster {}: {}\n".format(num + 1, ", ".join(words[word] for word in centroid)))